# preprocess dataset

In [1]:
# import json
# path_codex="/media/data/flowers/OpenELM/Codex_PAPER_1M_iter_0.txt"
# # with open(path_codex, 'r') as f:
# #     codex = f.read()
#     # codex = [x.strip() for x in codex]
# f = open(path_codex, 'r')
# out=f.read()
# f.close()

# path_save="/media/data/flowers/OpenELM/codex_dataset.json"
# def format_dataset(puzzles):
#     puzzles_prompt=""
#     for i in range(len(puzzles)):
#         puzzles_prompt += f"Puzzle {i}:\n"+puzzles[i]+"\n"

#     return puzzles_prompt
# n_fewshot=4
# dataset =[]
# split_out=out.split("\nassert f(g())\n\n")
# split_out = [x+"\nassert f(g())\n" for x in split_out if x]
# split_out = split_out[:-1]
# for idx in range(0,len(split_out),n_fewshot):
#     split_out_n_fs = split_out[idx:idx+n_fewshot]
#     if len(split_out_n_fs)==n_fewshot:
#         dataset.append(format_dataset(split_out_n_fs))
# dataset_json= [{'text': puzz} for puzz in dataset]
# with open(path_save, 'w') as outfile:
#     json.dump(dataset_json, outfile,indent=4)

In [ ]:
# test model
import os
os.environ['HF_DATASETS_CACHE'] = "/media/data/flowers/OpenELM/hf/datasets"
os.environ['TRANSFORMERS_CACHE'] = "/media/data/flowers/OpenELM/models"
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,TrainingArguments

from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer
from datasets import load_dataset

# base training

## train

In [1]:

# train on puzzle generated (except P3 trainset)
#https://huggingface.co/docs/trl/main/en/sft_trainer

path_train= "/media/data/flowers/OpenELM/logs/elm/23-08-22_12:36/step_9/save_all.json" # put path here
path_save="/media/data/flowers/OpenELM/codex_dataset.json"
dataset = load_dataset("json", data_files=path_save,split="train")
dataset = dataset.train_test_split(test_size=0.001)
# testset which is in fact the trainset from P3 dataset
# path_test = "/media/data/flowers/OpenELM/preprocess_p3.json"

 
# /!\ need to remove that, just for testing purpose /!\

# path_train = path_test 
# ================================================

# test_set = load_dataset("json", data_files=path_test)
model_id = "Salesforce/codegen-350M-mono"#"bigcode/starcoderbase-1b"  "Salesforce/codegen-350M-mono" # mono is better
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="bfloat16"#torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(model_id,quantization_config=bnb_config, device_map="auto",trust_remote_code=True)
model.config.use_cache = False



peft_config = LoraConfig(
    r=32,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["qkv_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)




===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /media/data/flowers/conda/envs/codegpt/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /media/data/flowers/conda/envs/codegpt/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /media/data/flowers/conda/envs/codegpt/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/media/data/flowers/conda/envs/codegpt/lib/python3.10/site-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
Using pad_token, but it is not set yet.
/media/data/flowers/conda/envs/codegpt/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:166: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/249749 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

/media/data/flowers/conda/envs/codegpt/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: julien_p (flowers-team). Use `wandb login --relogin` to force relogin


...

{'eval_loss': 0.6541656255722046, 'eval_runtime': 13.5956, 'eval_samples_per_second': 18.388, 'eval_steps_per_second': 3.089, 'epoch': 0.09}
{'loss': 0.7001, 'learning_rate': 3.939393939393939e-06, 'epoch': 0.09}
{'loss': 0.6563, 'learning_rate': 3.8383838383838385e-06, 'epoch': 0.09}
{'loss': 0.6958, 'learning_rate': 3.7373737373737375e-06, 'epoch': 0.09}
{'loss': 0.6391, 'learning_rate': 3.636363636363636e-06, 'epoch': 0.09}
{'loss': 0.7471, 'learning_rate': 3.5353535353535352e-06, 'epoch': 0.09}
{'loss': 0.7052, 'learning_rate': 3.4343434343434343e-06, 'epoch': 0.09}
{'loss': 0.7305, 'learning_rate': 3.3333333333333333e-06, 'epoch': 0.09}
{'loss': 0.6776, 'learning_rate': 3.2323232323232324e-06, 'epoch': 0.09}
{'loss': 0.582, 'learning_rate': 3.1313131313131314e-06, 'epoch': 0.09}
{'loss': 0.6363, 'learning_rate': 3.0303030303030305e-06, 'epoch': 0.09}


  0%|          | 0/42 [00:00<?, ?it/s]

{'eval_loss': 0.6541017889976501, 'eval_runtime': 13.9795, 'eval_samples_per_second': 17.883, 'eval_steps_per_second': 3.004, 'epoch': 0.09}
{'loss': 0.7174, 'learning_rate': 2.9292929292929295e-06, 'epoch': 0.09}
{'loss': 0.6507, 'learning_rate': 2.8282828282828286e-06, 'epoch': 0.09}
{'loss': 0.6907, 'learning_rate': 2.7272727272727272e-06, 'epoch': 0.09}
{'loss': 0.6287, 'learning_rate': 2.6262626262626263e-06, 'epoch': 0.09}
{'loss': 0.6873, 'learning_rate': 2.5252525252525253e-06, 'epoch': 0.09}
{'loss': 0.6671, 'learning_rate': 2.4242424242424244e-06, 'epoch': 0.09}
{'loss': 0.7106, 'learning_rate': 2.3232323232323234e-06, 'epoch': 0.09}
{'loss': 0.6402, 'learning_rate': 2.2222222222222225e-06, 'epoch': 0.1}
{'loss': 0.6703, 'learning_rate': 2.1212121212121216e-06, 'epoch': 0.1}
{'loss': 0.6258, 'learning_rate': 2.0202020202020206e-06, 'epoch': 0.1}


  0%|          | 0/42 [00:00<?, ?it/s]

KeyboardInterrupt: 

# train

In [ ]:
# def formatting_func(example):
#     # text = f"### Question: {example['question']}\n ### Answer: {example['answer']}"
#     text = example["text"]
#     return text


training_arguments=TrainingArguments(
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    evaluation_strategy="steps",
    gradient_accumulation_steps=2,
    do_eval=True,
    # warmup_steps=2,
    warmup_ratio=0.01,
    max_steps = 2000,
    eval_steps=10,
    weight_decay=0.001,
    learning_rate=2e-4,
    
    # fp16=True,
    logging_steps=1,
    output_dir="outputs",
    optim="adamw_hf",#"paged_adamw_32bit",
    max_grad_norm=0.3,
    group_by_length=True
)

trainer = SFTTrainer(
    model,#"EleutherAI/gpt-neo-125m",
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    dataset_text_field="text",
    peft_config=peft_config,
    args= training_arguments
)

trainer.train()
model.config.use_cache = True

In [3]:
output_dir = "/media/data/flowers/OpenELM/models/codegen-QDAIF-smart"
trainer.save_model(output_dir)
from peft import AutoPeftModelForCausalLM
del model
model = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map="auto", torch_dtype=torch.bfloat16)
model = model.merge_and_unload()
model.eval()
model.config.use_cache = True

# test

In [1]:
def test_puzzle(test_fg):
    try:
        exec(test_fg)
        return True
    except:
        return False
    
prompt_solve_puzzle='''You will be given a function and its docstring. Respond only in code with a correct, efficient implementation of the function.
You need to generate the correct solutions (g), for the Problem 3 that satisfies the condition f(g()) == True.
Problem 0:
```
def f(stamps: List[int], target=80, max_stamps=4, options=[10, 32, 8]) -> bool:
    """Find a selection of at most max_stamps stamps whose total worth is the target value."""
    for s in stamps:
        assert s in options
    return len(stamps) <= max_stamps and sum(stamps) == target
```
Solution 0:
```
def g(target = 80, max_stamps = 4, options = [10, 32, 8]):
    from itertools import combinations_with_replacement
    for n in range(max_stamps + 1):
        for c in combinations_with_replacement(options, n):
            if sum(c) == target:
                return list(c)
assert f(g())
```
Problem 1:
```
from typing import*
def f(ans: List[List[int]], target=2) -> bool:
    """
    Find a list of pairs of integers where the number of pairs in which the second number is more than
    two greater than the first number is a given constant
    """
    for i in range(len(ans)):
        a, b = ans[i]
        if b - a >= 2:
            target -= 1
    return target == 0
```
Solution 1:
```
def g(target = 2):
    return [[0, 2]] * target 
assert f(g()) == True
```
Problem 2:
```
def f(n: int, v=313946483, w=806690290) -> bool:
    """Find the smallest n such that if v is tripled n times and w is doubled n times, v exceeds w."""
    for i in range(n):
        assert v <= w
        v *= 3
        w *= 2
    return v > w
```
Solution 2:
```
def g(v = 313946483, w = 806690290):
    i = 0
    while v <= w:
        v *= 3
        w *= 2
        i += 1
    return i 
assert f(g()) == True
```
Problem 3:
```
{pb}
```
Solution 3:
```
def g('''

In [6]:
model_id="TheBloke/CodeLlama-13B-Python-fp16"

quantization_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_id,trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto",
    trust_remote_code=True
)
model.config.use_cache = False

prompt = 'def remove_non_ascii(s: str) -> str:\n    """ '
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

output = model.generate(
    inputs["input_ids"],
    max_new_tokens=200,
    do_sample=True,
    top_p=0.9,
    temperature=0.1,
)
output = output[0].to("cpu")
print(tokenizer.decode(output))




In [ ]:
import json
path_trainset="/media/data/flowers/OpenELM/src/openelm/utils/preprocess_p3_emb.json"
with open(path_trainset, 'r') as f:
    trainset = json.load(f)
    
list_trainset= [x["program_str"] for x in trainset]


In [8]:
correct_puzz=0
with torch.no_grad():
    list_puzzle=[]
    for idx in range(20): #len(dataset["test"])
        print(f"\n\n============ idx {idx} \n")
        puzzle= list_trainset[idx]
        prompt_f = puzzle.split("def g(")[0]
        prompt = prompt_solve_puzzle.format(pb=prompt_f)
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), attention_mask=inputs["attention_mask"], max_new_tokens=256,do_sample=True, temperature=0.8,num_return_sequences=2)
        len_prompt=inputs["input_ids"][0].shape[0]
        cor_puz=0
        for i in range(2):
            out ="def g("
            out += tokenizer.decode(outputs[i][len_prompt:], skip_special_tokens=True)

            list_puzzle.append(prompt_f+out)
            extract_g=out.split("assert")[0]
            extract_g = extract_g+"\nassert f(g())\n"
            test_fg= prompt_f+extract_g 
            print(test_fg)
            if test_puzzle(test_fg):
                cor_puz+=1
        if cor_puz>=1:
            print("correct puzzle",correct_puzz)
            correct_puzz+=1

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.




============ idx 0 



Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


def f(s: str) -> bool:
    """Find a string with 1000 'o's but no two adjacent 'o's."""
    return s.count('o') == 1000 and s.count('oo') == 0
def g( s = "This is a string with 1000 'o's but no two adjacent 'o's.":
         s.count('o') == 1000 and s.count('oo') == 0):
         return s.replace('o', 'oo')
```
assert f(g())

def f(s: str) -> bool:
    """Find a string with 1000 'o's but no two adjacent 'o's."""
    return s.count('o') == 1000 and s.count('oo') == 0
def g(	s: str,
        	target: str = "1000o",
        	count_before: int = 1000,
        	count_after: int = 0,
        	max_count: int = 1000,
        ) -> str:
            for i in range(1, len(s) + 1):
                if i > max_count:
                    break
                s += "o"
            else:
                s = target + count_after * "o"
            return s
        
assert f(g())



============ idx 1 



Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


def f(li: List[int]) -> bool:
    """
    Find a list of integers whose pairwise sums make the set {0, 1, 2, 3, 4, 5, 6, 17, 18, 19, 20, 34}.
    That is find L such that, { i + j | i, j in L } = {0, 1, 2, 3, 4, 5, 6, 17, 18, 19, 20, 34}.
    """
    return {i + j for i in li for j in li} == {0, 1, 2, 3, 4, 5, 6, 17, 18, 19, 20, 34}
def g( li: List[int]
        ) -> List[int]:
    """
    Return a list of integers whose pairwise sums make the set {0, 1, 2, 3, 4, 5, 6, 17, 18, 19, 20, 34}.
    That is find L such that, { i + j | i, j in L } = {0, 1, 2, 3, 4, 5, 6, 17, 18, 19, 20, 34}.
    """
    return {i + j for i in li for j in li}

assert f(g())

def f(li: List[int]) -> bool:
    """
    Find a list of integers whose pairwise sums make the set {0, 1, 2, 3, 4, 5, 6, 17, 18, 19, 20, 34}.
    That is find L such that, { i + j | i, j in L } = {0, 1, 2, 3, 4, 5, 6, 17, 18, 19, 20, 34}.
    """
    return {i + j for i in li for j in li} == {0, 1, 2, 3, 4, 5, 6, 17, 18, 19, 20, 34}
def g(	

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


def f(li: List[int]) -> bool:
    """
    Find a list integers containing exactly three distinct values, such that no integer repeats
    twice consecutively among the first eleven entries. (So the list needs to have length greater than ten.)
    """
    return all([li[i] != li[i + 1] for i in range(10)]) and len(set(li)) == 3
def g( target=313946483, 
         max_stamps=4, 
         options=[10, 32, 8], 
         count_elements=[2, 5, 10], 
         count_elements_set=set(count_elements), 
         count_elements_set_size=len(count_elements_set)
        ) -> bool:
            """
            Find a selection of at most max_stamps stamps whose total worth is the target value.
            """
            for n in range(max_stamps + 1):
                for c in combinations_with_replacement(options, n):
                    if all([sum(c) == target for s in count_elements_set.difference(set(c))]):
                        return n
            return False
        
assert f(g())

def f(li:

KeyboardInterrupt: 

In [6]:
correct_puzz=0
with torch.no_grad():
    
    for idx in range(10): #len(dataset["test"])
        text = dataset["test"][idx]["text"]
        split_text=text.split("Puzzle 3:")
        prompt_f = split_text[1].split("def g(")[0]
        prompt = split_text[0]+"Puzzle 3:"+prompt_f
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), attention_mask=inputs["attention_mask"], max_new_tokens=128,do_sample=True, temperature=0.9)
        len_prompt=inputs["input_ids"][0].shape[0]
        out = tokenizer.decode(outputs[0][len_prompt:], skip_special_tokens=True)
        # out= tokenizer.decode(outputs[0], skip_special_tokens=True)
        print("\n\n============ idx ",idx)
        
        extract_g=out.split("assert")[0]
        extract_g = extract_g+"\nassert f(g())\n"
        test_fg= prompt_f+extract_g 
        print(test_fg)
        if test_puzzle(test_fg):
            correct_puzz+=1
            

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




============ idx  0

def f(n: int, pack=15):
    return n >= pack


def g(pack=15):
    return sum(i for i in range(pack, 0, -1) if isinstance(i, int)) + int(pack ** 2)


assert f(g())



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




============ idx  1

def f(multiples: List[int], bound=100):
    return multiples == sorted(multiples, reverse=True) and sum(multiples) > bound * 2


def g(bound=100):
    return sorted(range(bound), reverse=True)


assert f(g())



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...



============ idx  6

def f(cols: List[int], m=8, n=8):
    assert set(cols) == set(range(n)) and len(set([i - cols[i] for i in range(m)])) == 1 and (
            (len(set(cols)) == n and cols[0] == 0) or (len(set(cols)) == n - 1 and cols[0] == -1))
    return len(set(cols)) == n == m


def g(m=8, n=8):
    cols = list(range(m))
    return cols[1:] + cols[:-1]


assert f(g())



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




============ idx  7

def f(nums: List[int], t=369601, n=5):
    return sum(1 for num in nums if num < t) == n


def g(t=369601, n=5):
    return [t] * n


assert f(g())



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




============ idx  8

def f(magic: List[int], start=173673503):
    return magic[0] == start and all(magic[i] < magic[i + 1] for i in range(len(magic) - 1))


def g(start=173673503):
    return [start, start, start, start]


assert f(g())



============ idx  9

def f(s: str, lower=100, upper=80000):
    return len(s) >= lower and len(s) <= upper and s[0] == "f" and (s.count("t") + s.count("T")) == s.count("o") + s.count("O")


def g(lower=100, upper=80000):
    return "fT" * (upper - lower) + "o" * upper


assert f(g())



In [7]:
correct_puzz

2

296

# train only on completion

In [ ]:
import os
os.environ['TRANSFORMERS_CACHE'] = "/media/data/flowers/OpenELM/models"
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from datasets import load_dataset

# train on puzzle generated (except P3 trainset)
#https://huggingface.co/docs/trl/main/en/sft_trainer


path_train= "/media/data/flowers/OpenELM/logs/elm/23-08-22_12:36/step_9/save_all.json" # put path here
dataset = load_dataset("json", data_files=path_train, split="train")


model_id = "Salesforce/codegen2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="bfloat16"#torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(model_id,quantization_config=bnb_config, device_map="auto",trust_remote_code=True)
model.config.use_cache = False
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

import json



path = "/media/data/flowers/OpenELM/preprocess_p3.json" 

# path = "/media/data/flowers/OpenELM/logs/elm/23-08-22_12:36/step_9/save_all.json" # QDAIF smart
with open(path, 'r') as f:
    data = json.load(f)
f=data[0]["program_str"].split("def g")[0]
prompt=""
n_fewshot=3
for i in range(1,1+n_fewshot):
    prompt+=f"Puzzle {i}:\n"+data[i]["program_str"]+"\n\n"
# text = prompt+ f"Puzzle {n_fewshot}:\n"+ f +"\ndef g():"

def formatting_prompts_func(example,prompt=prompt):
    output_texts = []
    text = f"{prompt}\nPuzzle 4:\n {example['program_str'][i]}"
    output_texts.append(text)
    return output_texts

response_template = "Puzzle 4:\n"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)


tokenizer.padding_side = "right"


training_arguments=TrainingArguments(
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    # evaluation_strategy="epoch", #"steps",
    gradient_accumulation_steps=4,
    # warmup_steps=2,
    warmup_ratio=0.2,
    max_steps = 12,
    weight_decay=0.001,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=1,
    output_dir="outputs",
    optim="adamw_hf",#"paged_adamw_32bit",
    max_grad_norm=0.3,
    # group_by_length=True
)

trainer = SFTTrainer(
    model,#"EleutherAI/gpt-neo-125m",
    train_dataset=dataset,
    formatting_func=formatting_prompts_func,
    data_collator=collator,
    peft_config=peft_config,
    args=training_arguments
)

trainer.train()

output_dir = "/media/data/flowers/OpenELM/models/codegen-QDAIF-smart"
trainer.model.save_pretrained(output_dir)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /media/data/flowers/conda/envs/codegpt/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /media/data/flowers/conda/envs/codegpt/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /media/data/flowers/conda/envs/codegpt/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/media/data/flowers/conda/envs/codegpt/lib/python3.10/site-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
Using pad_token, but it is not set yet.
/media/data/flowers/conda/envs/codegpt/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:166: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/media/data/flowers/conda/envs/codegpt/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb:

  0%|          | 0/12 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/julien/.cache/huggingface/modules/transformers_modules/Salesforce/codegen2-1B/4a7f51462b80ac8086d75c54eeb4851754833c41/modeling_codegen.py:167: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995026/work/aten/src/ATen/native/TensorCompare.cpp:493.)
  attn_weights = torch.where(causal_mask, attn_weights, mask_value)


{'loss': 0.1843, 'learning_rate': 6.666666666666667e-05, 'epoch': 1.0}
{'loss': 0.1843, 'learning_rate': 0.00013333333333333334, 'epoch': 2.0}
{'loss': 0.1813, 'learning_rate': 0.0002, 'epoch': 3.0}
{'loss': 0.1741, 'learning_rate': 0.00017777777777777779, 'epoch': 4.0}
{'loss': 0.163, 'learning_rate': 0.00015555555555555556, 'epoch': 5.0}
{'loss': 0.1528, 'learning_rate': 0.00013333333333333334, 'epoch': 6.0}
{'loss': 0.1434, 'learning_rate': 0.00011111111111111112, 'epoch': 7.0}
{'loss': 0.1348, 'learning_rate': 8.888888888888889e-05, 'epoch': 8.0}
{'loss': 0.1272, 'learning_rate': 6.666666666666667e-05, 'epoch': 9.0}
{'loss': 0.1209, 'learning_rate': 4.4444444444444447e-05, 'epoch': 10.0}
{'loss': 0.1161, 'learning_rate': 2.2222222222222223e-05, 'epoch': 11.0}
{'loss': 0.1124, 'learning_rate': 0.0, 'epoch': 12.0}
{'train_runtime': 12.2257, 'train_samples_per_second': 15.705, 'train_steps_per_second': 0.982, 'train_loss': 0.14953618682920933, 'epoch': 12.0}


CodeGenForCausalLM(
  (transformer): CodeGenModel(
    (wte): Embedding(51200, 2048)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-15): 16 x CodeGenBlock(
        (ln_1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (attn): CodeGenAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (qkv_proj): Linear4bit(
            in_features=2048, out_features=6144, bias=False
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2048, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=6144, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (out_proj): Linear4bit(in

In [ ]:
import numpy as np
out=[15,17,16.5,16.,16.,17,16.25,17.3,15]
len(out)
out.sort(reverse=True)
np.mean(out[:8])

16.38125

In [ ]:
import json
path = "/media/data/flowers/OpenELM/preprocess_p3.json" 

# path = "/media/data/flowers/OpenELM/logs/elm/23-08-22_12:36/step_9/save_all.json" # QDAIF smart
with open(path, 'r') as f:
    data = json.load(f)

f=data[0]["program_str"].split("def g")[0]
prompt=""
n_fewshot=3
for i in range(1,1+n_fewshot):
    prompt+=f"Puzzle {i}:\n"+data[i]["program_str"]+"\n\n"
    # prompt+=data[i]["program_str"]+"\n"
tag_response = f"Puzzle {n_fewshot+1}:\n"
tag_eo_completion = f"Puzzle"
text = prompt+ f"Puzzle {n_fewshot+1}:\n"+ f #+"def g():"
# text = f#prompt+ f 


with torch.no_grad():
    inputs = tokenizer(text, return_tensors="pt").to("cuda")
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), attention_mask=inputs["attention_mask"], max_new_tokens=256,do_sample=True,temperature=1.)
    gen_text= tokenizer.decode(outputs[0], skip_special_tokens=True)
    print()

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
model

CodeGenForCausalLM(
  (transformer): CodeGenModel(
    (wte): Embedding(51200, 2048)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-15): 16 x CodeGenBlock(
        (ln_1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (attn): CodeGenAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (qkv_proj): Linear4bit(
            in_features=2048, out_features=6144, bias=False
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2048, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=6144, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (out_proj): Linear4bit(in

In [ ]:
gen_text.split(tag_response)[1].split(tag_eo_completion)[0].strip()

'def f(s: str) -> bool:\n    """Find a string with 1000 \'o\'s but no two adjacent \'o\'s."""\n    return s.count(\'o\') == 1000 and s.count(\'oo\') == 0\ndef g():\n    s = \'ooyo\'\n    return \'\'.join(s).count(\'o\')\nassert f(g())'

In [ ]:
texts = [text]*5
encoding = tokenizer(texts, padding=True, return_tensors='pt').to("cuda")
with torch.no_grad():
    generated_ids = model.generate(**encoding,max_new_tokens=256,do_sample=True,temperature=1.)
generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
filter_text = [puzz.split(tag_response)[1].split(tag_eo_completion)[0].strip() for puzz in generated_texts]

In [ ]:
[print(puzz+"\n") for puzz in filter_text]

def f(s: str) -> bool:
    """Find a string with 1000 'o's but no two adjacent 'o's."""
    return s.count('o') == 1000 and s.count('oo') == 0
def g():
    return [i for i in range(1000) if (2*i,2*i+1) in sorted(range(1000))]
assert f(g())

def f(s: str) -> bool:
    """Find a string with 1000 'o's but no two adjacent 'o's."""
    return s.count('o') == 1000 and s.count('oo') == 0
def g():
    s = 'ooxo'
    return ''.join(s + 'o' if i % 2 == 0 else 'ooo' for i in range(3*4*5))
assert f(g())

def f(s: str) -> bool:
    """Find a string with 1000 'o's but no two adjacent 'o's."""
    return s.count('o') == 1000 and s.count('oo') == 0
def g():
    return [*(s for s in open('input10') if s.count('o') > 1), 'oo']
assert f(g())

def f(s: str) -> bool:
    """Find a string with 1000 'o's but no two adjacent 'o's."""
    return s.count('o') == 1000 and s.count('oo') == 0
def g():
    s = 'o'*1000 + 'oo'
    return s
assert f(g())

def f(s: str) -> bool:
    """Find a string with 1000 'o's but

[None, None, None, None, None]

# other stuff

In [ ]:
import os
os.environ['TRANSFORMERS_CACHE'] = "/media/data/flowers/OpenELM/models"
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
# from huggingface_hub import notebook_login
# notebook_login()

peft=True
model_id = "Salesforce/codegen-350M-mono"#"bigcode/starcoderbase-1b"#"Salesforce/codegen-350M-mono" # mono is better
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"#torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
if peft:
    model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto",trust_remote_code=True)
else:
    model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto",trust_remote_code=True)
    
if peft:
    # model.gradient_checkpointing_enable()
    model = prepare_model_for_kbit_training(model)

    def print_trainable_parameters(model):
        """
        Prints the number of trainable parameters in the model.
        """
        trainable_params = 0
        all_param = 0
        for _, param in model.named_parameters():
            all_param += param.numel()
            if param.requires_grad:
                trainable_params += param.numel()
        print(
            f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
        )
    print(model)


    config = LoraConfig(
        r=16, 
        lora_alpha=32, 
        target_modules=["qkv_proj"],# "lm_head" s["c_attn","c_proj"],# codegen ["qkv_proj"],#["query_key_value"],  
        lora_dropout=0.01, 
        bias="none", 
        task_type="CAUSAL_LM",
        # layers_to_transform=[i for i in range(5,15)]
    )

    model = get_peft_model(model, config)
    print_trainable_parameters(model)

from datasets import load_dataset
# train on puzzle generated (except P3 trainset)

path_train= "/media/data/flowers/OpenELM/logs/elm/23-08-22_12:36/step_9/save_all.json" # put path here
squad_it_dataset = load_dataset("json", data_files=path_train)

# testset which is in fact the trainset from P3 dataset
path_test = "/media/data/flowers/OpenELM/preprocess_p3.json"

 
# /!\ need to remove that, just for testing purpose /!\

path_train = path_test 
# ================================================

test_set = load_dataset("json", data_files=path_test)

def filter_correct_puzz(example):
    return (
        example["fitness"] >0.
    )
# only keep correct puzzles
correct_dataset = squad_it_dataset.filter(filter_correct_puzz)

tokenized_datasets = correct_dataset.map(lambda samples: tokenizer(samples["program_str"]), batched=True)
tok_test_set = test_set.map(lambda samples: tokenizer(samples["program_str"]), batched=True)

CodeGenForCausalLM(
  (transformer): CodeGenModel(
    (wte): Embedding(51200, 1024)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-19): 20 x CodeGenBlock(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): CodeGenAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (qkv_proj): Linear4bit(in_features=1024, out_features=3072, bias=False)
          (out_proj): Linear4bit(in_features=1024, out_features=1024, bias=False)
        )
        (mlp): CodeGenMLP(
          (fc_in): Linear4bit(in_features=1024, out_features=4096, bias=True)
          (fc_out): Linear4bit(in_features=4096, out_features=1024, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=51200, bi

In [ ]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_datasets["train"],
    eval_dataset = tok_test_set,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        evaluation_strategy="epoch", #"steps",
        gradient_accumulation_steps=2,
        # warmup_steps=2,
        warmup_ratio=0.1,
        num_train_epochs = 2,
        # max_steps=10,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_32bit",
        max_grad_norm=0.3,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: julien_p (flowers-team). Use `wandb login --relogin` to force relogin


  0%|          | 0/4 [00:00<?, ?it/s]

You're using a CodeGenTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.769, 'learning_rate': 0.0002, 'epoch': 0.4}
{'loss': 0.8553, 'learning_rate': 0.00013333333333333334, 'epoch': 0.8}


  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_train_loss': 1.735876441001892, 'eval_train_runtime': 0.3282, 'eval_train_samples_per_second': 60.948, 'eval_train_steps_per_second': 15.237, 'epoch': 0.8}
{'loss': 0.8522, 'learning_rate': 6.666666666666667e-05, 'epoch': 1.2}
{'loss': 0.7343, 'learning_rate': 0.0, 'epoch': 1.6}


  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_train_loss': 1.7316404581069946, 'eval_train_runtime': 0.3495, 'eval_train_samples_per_second': 57.218, 'eval_train_steps_per_second': 14.304, 'epoch': 1.6}
{'train_runtime': 12.2976, 'train_samples_per_second': 3.09, 'train_steps_per_second': 0.325, 'train_loss': 0.8026903122663498, 'epoch': 1.6}


TrainOutput(global_step=4, training_loss=0.8026903122663498, metrics={'train_runtime': 12.2976, 'train_samples_per_second': 3.09, 'train_steps_per_second': 0.325, 'train_loss': 0.8026903122663498, 'epoch': 1.6})

In [ ]:
path_model = "/media/data/flowers/OpenELM/models/codegen-QDAIF-smart"
trainer.model.save_pretrained(path_model)


In [ ]:
from peft import LoraConfig, PeftModel

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
model = PeftModel.from_pretrained(base_model, path_model)
model = model.merge_and_unload()



In [ ]:
from transformers import pipeline
import json



path = "/media/data/flowers/OpenELM/preprocess_p3.json" 

# path = "/media/data/flowers/OpenELM/logs/elm/23-08-22_12:36/step_9/save_all.json" # QDAIF smart
with open(path, 'r') as f:
    data = json.load(f)
f=data[0]["program_str"].split("def g")[0]
prompt=""
n_fewshot=2
for i in range(1,n_fewshot):
    prompt+=f"Puzzle {i}:\n"+data[i]["program_str"]+"\n\n"
text = prompt+ f"Puzzle {n_fewshot}:\n"+ f +"\ndef g():"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=512)
result = pipe(text)
print(result[0]['generated_text'])

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Puzzle 1:
def f(li: List[int]) -> bool:
    """Find a permutation of [0, 1, ..., 998] such that the ith element is *not* i, for all i=0, 1, ..., 998."""
    return sorted(li) == list(range(999)) and all((li[i] != i for i in range(len(li))))
def g():
    return [((i + 1) % 999) for i in range(999)]
assert f(g())

Puzzle 2:
def f(s: str) -> bool:
    """Find a string with 1000 'o's but no two adjacent 'o's."""
    return s.count('o') == 1000 and s.count('oo') == 0

o)formpolytokenhandle++,#form(vc<mask_1>�In,porttexto<mask_1>))(vt(<mask_18><form<mask_1>%stform...(...port(<mask_18>escapepolyo(ob(form;poly<mask_18>in varformtext("),st(hasInescape"),(<mask_1>->input,<mask_52>/(++(st(wisevt<port(W'),�<wisepoly<mask_1>var


In [ ]:
import json
path = "/media/data/flowers/OpenELM/preprocess_p3.json" 

# path = "/media/data/flowers/OpenELM/logs/elm/23-08-22_12:36/step_9/save_all.json" # QDAIF smart
with open(path, 'r') as f:
    data = json.load(f)

f=data[11]["program_str"].split("def g")[0]
prompt=""
n_fewshot=1
for i in range(n_fewshot):
    prompt+=f"Puzzle {i}:\n"+data[i]["program_str"]+"\n\n"
text = prompt+ f"Puzzle {n_fewshot}:\n"+ f #+"def g():"
# text = f


with torch.no_grad():
    inputs = tokenizer(text, return_tensors="pt").to("cuda")
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), attention_mask=inputs["attention_mask"], max_new_tokens=256)

    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


def f(ls: List[str]) -> bool:
    """
    Find a list of characters which are aligned at the same indices of the three strings 'dee', 'doo', and 'dah!'.
    """
    return tuple(ls) in zip('dee', 'doo', 'dah!')
(/vt(/vt(/vt(/vt(/vt(/vt(/(/(/(/(/(input(input(input(input(input(input(input(input(input(input(input(input(input(input(input(input(input(input(input(input(input(input(input(input(input(input(input(input(input(input(input(input(input(input(input(input(input,input(input(input,input,input,input,input,input,input,input,input,input,input,input(input,input,input,input(input,input,input,input,input,input,input,input,input,input(input,input,input,input(input,input(input,input(input(input(input(input(input(input(input(input(input(input,input,input,input,input(input,input(input(input(input(input(input,input,input(input,input(input(input,input(input,input(input,input,input,input,input,input(input,input,input,input,input


In [ ]:
model_id = "Salesforce/codegen2-1B"#"bigcode/starcoderbase-1b"#"Salesforce/codegen-350M-mono" # mono is better
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto",trust_remote_code=True)
f=data[11]["program_str"].split("def g")[0]
prompt=""
n_fewshot=2
for i in range(n_fewshot):
    prompt+=f"Puzzle {i}:\n"+data[i]["program_str"]+"\n\n"
text = prompt+ f"Puzzle {n_fewshot}:\n"+ f #+"def g():"
with torch.no_grad():
    inputs = tokenizer(text, return_tensors="pt").to("cuda")
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), attention_mask=inputs["attention_mask"], max_new_tokens=256)

    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Puzzle 0:
def f(s: str) -> bool:
    """Find a string with 1000 'o's but no two adjacent 'o's."""
    return s.count('o') == 1000 and s.count('oo') == 0
def g():
    return ('h' + 'o') * 1000
assert f(g())

Puzzle 1:
def f(li: List[int]) -> bool:
    """Find a permutation of [0, 1,..., 998] such that the ith element is *not* i, for all i=0, 1,..., 998."""
    return sorted(li) == list(range(999)) and all((li[i]!= i for i in range(len(li))))
def g():
    return [((i + 1) % 999) for i in range(999)]
assert f(g())

Puzzle 2:
def f(ls: List[str]) -> bool:
    """
    Find a list of characters which are aligned at the same indices of the three strings 'dee', 'doo', and 'dah!'.
    """
    return tuple(ls) in zip('dee', 'doo', 'dah!')
def g():
    return 'dee' + 'doo' + 'dah!'
assert f(g())

Puzzle 3:
def f(ls: List[str]) -> bool:
    """
    Find a list of characters which are aligned at the same indices of the three strings 'dee', 'doo', and 'dah!'.
    """
    return tuple(ls) in zip('dee